In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [1]:
import os, cv2, sklearn
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
from keras import backend as K
from keras.regularizers import l2
from keras.layers import LeakyReLU
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.utils import np_utils

Using TensorFlow backend.


In [4]:
root_dir = os.getcwd() + "/drive/road_extract/data/sample/"
image_dir = root_dir + "train_sample/"
mask_dir = root_dir + "mask_sample/"

imgs, mask_imgs = [], []


for im in os.listdir(image_dir):
  imPath = image_dir + im
  maskPath = mask_dir + im.split('_')[0] + "_mask.png"
  
  imgs.append(load_image(imPath))
  mask_imgs.append(load_gray_image(maskPath))
  
  print(imPath, maskPath)


imgs = np.asarray(imgs)
mask_imgs = np.asarray(mask_imgs)

/content/drive/road_extract/data/sample/train_sample/104_sat.jpg /content/drive/road_extract/data/sample/mask_sample/104_mask.png
/content/drive/road_extract/data/sample/train_sample/541_sat.jpg /content/drive/road_extract/data/sample/mask_sample/541_mask.png
/content/drive/road_extract/data/sample/train_sample/388_sat.jpg /content/drive/road_extract/data/sample/mask_sample/388_mask.png
/content/drive/road_extract/data/sample/train_sample/627_sat.jpg /content/drive/road_extract/data/sample/mask_sample/627_mask.png
/content/drive/road_extract/data/sample/train_sample/655_sat.jpg /content/drive/road_extract/data/sample/mask_sample/655_mask.png
/content/drive/road_extract/data/sample/train_sample/602_sat.jpg /content/drive/road_extract/data/sample/mask_sample/602_mask.png
/content/drive/road_extract/data/sample/train_sample/113_sat.jpg /content/drive/road_extract/data/sample/mask_sample/113_mask.png
/content/drive/road_extract/data/sample/train_sample/951_sat.jpg /content/drive/road_extra

/content/drive/road_extract/data/sample/train_sample/12810_sat.jpg /content/drive/road_extract/data/sample/mask_sample/12810_mask.png
/content/drive/road_extract/data/sample/train_sample/12347_sat.jpg /content/drive/road_extract/data/sample/mask_sample/12347_mask.png
/content/drive/road_extract/data/sample/train_sample/12023_sat.jpg /content/drive/road_extract/data/sample/mask_sample/12023_mask.png
/content/drive/road_extract/data/sample/train_sample/11168_sat.jpg /content/drive/road_extract/data/sample/mask_sample/11168_mask.png
/content/drive/road_extract/data/sample/train_sample/14361_sat.jpg /content/drive/road_extract/data/sample/mask_sample/14361_mask.png
/content/drive/road_extract/data/sample/train_sample/15347_sat.jpg /content/drive/road_extract/data/sample/mask_sample/15347_mask.png
/content/drive/road_extract/data/sample/train_sample/13635_sat.jpg /content/drive/road_extract/data/sample/mask_sample/13635_mask.png
/content/drive/road_extract/data/sample/train_sample/13870_sat

In [5]:
print(imgs.shape)
print(mask_imgs.shape)

(100, 1024, 1024, 3)
(100, 1024, 1024)


# Create Array of Windows and Labels

In [0]:
X = imgs
Y = mask_imgs

In [0]:
foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch

def value_to_class(v):
    df = np.sum(v)
    if df > foreground_threshold:
        return 1
    else:
        return 0

# Extract patches from input images
img_patches = [img_crop(X[i], 16, 16, 16, 0) for i in range(X.shape[0])]
gt_patches = [img_crop_gt(Y[i], 16, 16, 16) for i in range(X.shape[0])]

# Linearize list of patches
img_patches = np.asarray([img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))])
gt_patches =  np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])

X = np.asarray([img_patches[i] for i in range(len(img_patches))])
Y = np.asarray([value_to_class(np.mean(gt_patches[i])) for i in range(len(gt_patches))])

In [8]:
print(X.shape)
print(Y.shape)

(409600, 16, 16, 3)
(409600,)


In [0]:
Y_2d = np_utils.to_categorical(Y, num_classes=2)

In [14]:
print(np.count_nonzero(Y_2d[:,1]))
print(Y_2d.shape)
print(X.shape)

37857
(409600, 2)
(409600, 16, 16, 3)


# Pavllo Model

In [15]:
pool_size = (2, 2)
reg = 1e-6 # L2 regularization factor (used on weights, but not biases)
nb_classes = 2

model = Sequential()

model.add(Convolution2D(64, 5, 5, # 64 5x5 filters
                        border_mode='same',
                        input_shape=(16,16, 3)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=pool_size, border_mode='same'))
model.add(Dropout(0.25))

model.add(Convolution2D(128, 3, 3, # 128 3x3 filters
                        border_mode='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=pool_size, border_mode='same'))
model.add(Dropout(0.25))

model.add(Convolution2D(256, 3, 3, # 256 3x3 filters
                        border_mode='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=pool_size, border_mode='same'))
model.add(Dropout(0.25))

model.add(Convolution2D(256, 3, 3, # 256 3x3 filters
                        border_mode='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=pool_size, border_mode='same'))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, W_regularizer=l2(reg))) # Fully connected layer (128 neurons)
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.5))

model.add(Dense(nb_classes, W_regularizer=l2(reg)))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), input_shape=(16, 16, 3..., padding="same")`
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), padding="same")`
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), padding="same")`
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), padding="same")`
/us

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 16, 64)        4864      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 128)         73856     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 128)         0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 128)         0         
__________

In [0]:
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=adam,loss='binary_crossentropy',metrics=['accuracy'])

In [0]:
model.fit(X,Y_2d, batch_size=128, epochs=10)

Epoch 1/10
 69504/409600 [====>.........................] - ETA: 20:59 - loss: 1.4820 - acc: 2.1581e-05

# Helper Functions

In [0]:
def load_image(filename):
    img = cv2.imread(filename)
    resized_image = cv2.resize(img, (72,72)) 
    return img

def load_gray_image(infilename):
    img = cv2.imread(infilename)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    resized_image = cv2.resize(gray, (72,72)) 
    return gray

def pad_image(data, padding):
    """
    Extend the canvas of an image. Mirror boundary conditions are applied.
    """
    if len(data.shape) < 3:
        # Greyscale image (ground truth)
        data = np.lib.pad(data, ((padding, padding), (padding, padding)), 'reflect')
    else:
        # RGB image
        data = np.lib.pad(data, ((padding, padding), (padding, padding), (0,0)), 'reflect')
    return data
    
def img_crop_gt(im, w, h, stride):
    """ Crop an image into patches (this method is intended for ground truth images). """
    assert len(im.shape) == 2, 'Expected greyscale image.'
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    for i in range(0,imgheight,stride):
        for j in range(0,imgwidth,stride):
            im_patch = im[j:j+w, i:i+h]
            list_patches.append(im_patch)
    return list_patches
    
def img_crop(im, w, h, stride, padding):
    """ Crop an image into patches, taking into account mirror boundary conditions. """
    assert len(im.shape) == 3, 'Expected RGB image.'
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    im = np.lib.pad(im, ((padding, padding), (padding, padding), (0,0)), 'reflect')
    for i in range(padding,imgheight+padding,stride):
        for j in range(padding,imgwidth+padding,stride):
            im_patch = im[j-padding:j+w+padding, i-padding:i+h+padding, :]
            list_patches.append(im_patch)
    return list_patches
    
def create_patches(X, patch_size, stride, padding):
    img_patches = np.asarray([img_crop(X[i], patch_size, patch_size, stride, padding) for i in range(X.shape[0])])
    # Linearize list
    img_patches = img_patches.reshape(-1, img_patches.shape[2], img_patches.shape[3], img_patches.shape[4])
    return img_patches
    
def create_patches_gt(X, patch_size, stride):
    img_patches = np.asarray([img_crop_gt(X[i], patch_size, patch_size, stride) for i in range(X.shape[0])])
    # Linearize list
    img_patches = img_patches.reshape(-1, img_patches.shape[2], img_patches.shape[3])
    return img_patches
    
def group_patches(patches, num_images):
    return patches.reshape(num_images, -1)

def extract_img_features(filename, stride):
    img = load_image(filename)
    img_patches = img_crop(img, patch_size, patch_size, stride, padding)
    X = np.asarray([img_patches[i] for i in range(len(img_patches))])
    return X

def mask_to_submission_strings(model, image_filename):
    """ Reads a single image and outputs the strings that should go into the submission file. """
    img_number = int(re.search(r"\d+", image_filename).group(0))
    Xi = load_image(image_filename)
    Xi = Xi.reshape(1, Xi.shape[0], Xi.shape[1], Xi.shape[2])
    Zi = model.classify(Xi)
    Zi = Zi.reshape(-1)
    patch_size = 16
    nb = 0
    print("Processing " + image_filename)
    for j in range(0, Xi.shape[2], patch_size):
        for i in range(0, Xi.shape[1], patch_size):
            label = int(Zi[nb])
            nb += 1
            yield("{:03d}_{}_{},{}".format(img_number, j, i, label))